In [2]:
import pandas as pd
import numpy as np
import requests
from  data.pathsandmore import pathname, api_secret, api_id,app_Token
import functions 
from geopy.geocoders import Nominatim
import pickle
import playsound
import os
import re
import time

from datetime import datetime 
from datetime import timedelta
import importlib

importlib.reload(functions)


print("DONE")

DONE


In [4]:
df=pd.read_csv('batches.zip',compression='zip',parse_dates=['crash_date'])
start_date=str(df.crash_date.max())[:10]
print("START DATE: ",start_date)
#Where to end:
final_date=input("Enter new date 'yyyy-mm-dd'")
#final_date='2024-12-05'
final_date_dt= datetime.strptime(final_date, '%Y-%m-%d')
start_date_dt= datetime.strptime(start_date, '%Y-%m-%d')
#start_date=start_date.to_pydatetime()
new_date=start_date_dt+timedelta(1)
#Creating df to store data
new_df=pd.DataFrame()
while new_date <= final_date_dt:
    new_date_str=str(new_date)[:10]
    url=f'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$$app_token={app_Token}&crash_date={new_date_str}'
    resp=requests.get(url).json()
    print('RESP LEN:',len(resp),new_date_str)
    if new_df.empty:
        new_df=pd.DataFrame(resp)
        print(new_df.shape[0])
        new_date=new_date+timedelta(1)
    else:
        new_df=pd.concat([new_df,pd.DataFrame(resp)])
        print(new_date,new_df.shape[0])
        new_date=new_date+timedelta(1)

new_df=functions.adding_shp_boro(new_df)
new_df=functions.adding_shp_hood(new_df)

new_df=functions.adding_shp_zip(new_df)

#eliminate columns duplicated with zip code
zip_columns=[col for col in new_df.columns if 'zip_code' in col]
new_df[zip_columns[0]]=new_df[zip_columns].bfill(axis=1).iloc[:,[0]]
new_df.drop(zip_columns[1:],axis=1,inplace=True)

new_df=functions.adding_shp_st(new_df)

#next step is combnine this with batches.
rename_columns={'vehicle_type_code1':'vehicle_type_code_1',
               'vehicle_type_code2':'vehicle_type_code_2'}
new_df.rename(columns=rename_columns,inplace=True)
new_df.reset_index(inplace=True,drop=True)
print(df.shape,new_df.shape)
final_df=pd.concat([df, new_df])
final_df.to_csv('batches.zip',compression='zip',index=False)
functions.play_done_message()

START DATE:  2025-05-20


Enter new date 'yyyy-mm-dd' 2025-06-01


START DATE:  2025-05-20 
 FINAL DATE:  2025-06-01
2025-05-21 00:00:00 <class 'datetime.datetime'> 2025-05-20 00:00:00 2025-06-01 00:00:00
RESP LEN: 214
214
RESP LEN: 238
2025-05-22 00:00:00 452
RESP LEN: 253
2025-05-23 00:00:00 705
RESP LEN: 208
2025-05-24 00:00:00 913
RESP LEN: 204
2025-05-25 00:00:00 1117
RESP LEN: 175
2025-05-26 00:00:00 1292
RESP LEN: 253
2025-05-27 00:00:00 1545
RESP LEN: 238
2025-05-28 00:00:00 1783
RESP LEN: 220
2025-05-29 00:00:00 2003
RESP LEN: 249
2025-05-30 00:00:00 2252
RESP LEN: 212
2025-05-31 00:00:00 2464
RESP LEN: 181
2025-06-01 00:00:00 2645
1 (2645, 33)
2 (2645, 35)
3 (2645, 34)
4 (2645, 38)
(2177624, 38) (2645, 38)


<function functions.play_done_message()>